In [14]:
import numpy as np
import scipy as sp

d = 128

density = 0.1  # Density of non-zero elements (adjust as needed)
V = sp.sparse.random(d, d, density=density, format='coo')

# Ensure positive definiteness
V = (V + V.T) / 2

# Scale the matrix to have eigenvalues larger than 1
eigenvalues, eigenvectors = np.linalg.eig(V.toarray())

# Select eigenvalues less than or equal to 1
mask = np.abs(eigenvalues) <= 1

# Generate random numbers with absolute value larger than 1
random_numbers = np.random.uniform(1, 2, size=np.sum(mask))

# Preserve the original sign of eigenvalues
sign_mask = np.sign(eigenvalues[mask])
random_numbers *= sign_mask

# Update eigenvalues with the new values
eigenvalues[mask] = random_numbers

# Recreate the matrix using the updated eigenvalues
V = eigenvectors @ np.diag(eigenvalues) @ np.linalg.inv(eigenvectors)

# Convert the matrix to NumPy array format
V = np.array(V)

In [15]:
import cupy as np
from tqdm import tqdm

def get_dynamics(z_curr, attention, V, i):
    n = attention.shape[0]
    return np.sum(np.array([attention[i][j]*np.matmul(V, z_curr[j]-z_curr[i]) for j in range(n)]), axis=0)

V = np.array(V)

T = 15
dt = 0.1
n = 128

A = np.eye(d)

#    V = np.random.rand(d, d)
#    V = np.matmul(V, V.T)

#V = np.eye(d)
#V[-1, -1] = -0.5

(lams, phis) = np.linalg.eigh(V)
D = np.diag(lams)
Dexp = np.diag(np.exp(lams))

print("Eigenvalues of V:")
print(lams)

num_steps = int(T/dt)+1
z = np.zeros(shape=(n, num_steps, d), dtype=np.float64)
x0 = np.random.uniform(low=-0.5, high=0.5, size=(n, d))

z[:, 0, :] = np.matmul(phis.T, x0.T).T
integration_time = np.linspace(0, T, num_steps)

for l, t in tqdm(enumerate(integration_time[:-1]), total=num_steps-1):
    if l < num_steps - 1:
        # Attention matrix

        # on calcule z(t), qui est une matrice de shape (n, d)
        z_curr = z[:, l, :]
        #z_curr = np.matmul(phis.T, z_curr)

        exp_V_t = Dexp**t
        Bt = np.matmul(phis, np.matmul(exp_V_t, phis))
        z_exp = np.matmul(Bt, np.transpose(z_curr))           # Precompute A * exp(V * t) * z_curr

        z_diff = np.expand_dims(np.transpose(z_exp), axis=1) - np.expand_dims(np.transpose(z_exp), axis=0)  # Compute z_curr[j] - z_curr[i]
        dot_prod = np.matmul(np.expand_dims(np.transpose(z_exp), axis=0), np.expand_dims(z_diff, axis=-1))  # Compute dot product
        dot_prod_sum = np.sum(np.exp(dot_prod), axis=0)
        attention = np.reciprocal(np.concatenate(dot_prod_sum, axis=1))

        z_next = np.zeros((n, d))

        for i in range(n):
            k1 = dt * get_dynamics(z[:, l, :], attention, D, i)
            k2 = dt * get_dynamics(z[:, l, :] + k1 / 2, attention, D, i)
            k3 = dt * get_dynamics(z[:, l, :] + k2 / 2, attention, D, i)
            k4 = dt * get_dynamics(z[:, l, :] + k3, attention, D, i)

            z_next[i] = z[i][l] + (k1 + 2*k2 + 2*k3 + k4) / 6

        z[:, l+1, :] = z_next

Eigenvalues of V:
[-2.84568218 -2.60555879 -2.56283568 -2.42234939 -2.35907261 -2.28174486
 -2.19741075 -2.13233895 -2.1034233  -2.03243663 -2.02630259 -1.97773212
 -1.91690594 -1.91011697 -1.90334995 -1.90246591 -1.90085296 -1.89290549
 -1.86403407 -1.81274867 -1.80162854 -1.79143389 -1.76937543 -1.71738793
 -1.71598686 -1.7005023  -1.68964032 -1.64005423 -1.59352273 -1.55623747
 -1.55109089 -1.54870817 -1.54041316 -1.53434921 -1.52986133 -1.51749688
 -1.50540145 -1.48514417 -1.47944869 -1.45139727 -1.43012283 -1.39238451
 -1.39014349 -1.3736716  -1.35753497 -1.35375768 -1.34831195 -1.32586423
 -1.30012997 -1.2619633  -1.25573997 -1.24774612 -1.22271725 -1.20557957
 -1.19962132 -1.1548532  -1.15081277 -1.13130219 -1.12586656 -1.12556627
 -1.11004502 -1.09021636 -1.00697498  1.00657297  1.03955186  1.04492872
  1.05509063  1.06796254  1.08361658  1.08623141  1.0986496   1.12294856
  1.14328709  1.15225048  1.15361853  1.16704033  1.22397061  1.22405919
  1.27310654  1.28246594  1.31189

100%|██████████| 150/150 [32:00<00:00, 12.80s/it]


In [16]:
z[1, -1, :] - z[1, -10, :] 

array([-1.65871847e+11,  2.05887705e+12,  2.47818659e+12,  1.00156478e+12,
        6.06793691e+12,  1.94344763e+12,  1.36973975e+12, -1.69731876e+12,
        6.45955507e+10, -7.54630713e+10, -1.78484323e+11,  2.77034897e+10,
        5.68984713e+10, -3.54284485e+10,  3.01023017e+10,  5.38939841e+10,
       -6.27369681e+10, -1.89927834e+10,  7.96221077e+09,  9.39284689e+09,
       -7.70689875e+09, -1.08506219e+10,  1.45394326e+09, -2.01609489e+08,
        9.59079069e+08, -3.10316490e+09,  9.02446364e+08,  1.83350524e+09,
        1.67312526e+08, -2.64233387e+08, -2.22597681e+08,  1.25360500e+08,
        1.32043644e+08, -1.92155469e+08, -1.97790357e+07,  1.33250101e+07,
       -2.01517076e+08, -5.94786216e+07,  1.46156260e+08, -1.76626419e+08,
        6.11270194e+06,  2.88861224e+07,  2.28421743e+07, -1.85454897e+07,
        2.82074274e+06, -2.10260751e+07,  1.95189987e+07, -2.12663711e+06,
       -9.14334730e+06,  1.32320029e+05,  5.80520083e+06, -2.35954967e+06,
       -4.91570078e+06,  

In [17]:
z[1, -1, -1]

array(-0.51705436)

In [19]:
z[10, -1, -1]

array(-0.51705436)

In [21]:
z

array([[[-4.36634115e-02,  3.51862510e-01,  6.45458456e-03, ...,
          2.80325636e-01,  2.08541237e-01, -3.25261518e-01],
        [-3.18625568e-02,  3.35168365e-01, -2.16820124e-02, ...,
          3.11012120e-01,  2.37340687e-01, -7.66114677e-01],
        [-3.18111803e-02,  3.35439957e-01, -2.16913250e-02, ...,
          3.10833485e-01,  2.37173353e-01, -7.65476866e-01],
        ...,
        [-8.53237174e+11,  2.60507447e+13,  4.55892118e+13, ...,
          6.91814769e-02, -6.66427252e-02, -5.17054355e-01],
        [-1.52629071e+12,  4.40597906e+13,  7.63683917e+13, ...,
          6.91814769e-02, -6.66427252e-02, -5.17054355e-01],
        [-1.62804234e+12,  4.76861652e+13,  8.32039046e+13, ...,
          6.91814769e-02, -6.66427252e-02, -5.17054355e-01]],

       [[-2.53421505e-02,  1.42524775e-01,  6.10220584e-01, ...,
         -4.56339929e-01, -3.83423728e-01, -9.82435358e-02],
        [-1.69663179e-02,  7.87858824e-02,  7.35916973e-01, ...,
         -2.40413364e-01, -2.03236173e

In [30]:
# Convert the Cupy array to a NumPy array
#numpy_array = np.asnumpy(z)

 
# Save variable to a JSON file
with open('/kaggle/working/cupy_array_V.npy', 'wb') as file:
    np.save(file, V)
    
#z_load = numpy.load('/kaggle/working/cupy_array.npy')


In [ ]:
import json

z = np.asnumpy(z)

# Serialize variable to JSON
json_data = json.dumps(z.tolist())

%%writefile my_variable.json
{json_data}